# 🎯 AI Career Advisor Agent

An intelligent career transition assistant built with Google's Agent Development Kit (ADK).

## What This Agent Does

This agent helps people navigate career transitions by:
1. **Researching** target career paths (skills, resources, market outlook)
2. **Creating** personalized action plans (3-phase transition roadmap)

## Architecture

The system uses a **Sequential Agent** pattern:
```
User Query → Research Agent → Mentor Agent → Career Plan
```

**Research Agent**: Uses Google Search to gather information about career transitions
**Mentor Agent**: Creates personalized 3-phase action plans based on research

## Key Features

- ✅ **Memory Integration**: Remembers context across sessions
- ✅ **Structured Output**: Organized research and actionable plans
- ✅ **Automated Callbacks**: Saves conversations to memory automatically
- ✅ **Error Handling**: Graceful retry logic for API failures

## How to Use

1. Run all cells in order
2. Provide a career transition query (include: current role, experience, target role, time commitment)
3. Receive comprehensive research and a personalized action plan

**Example Query:**
> "I am a Product Manager with 5 years of experience. I want to transition to data science and can dedicate 10 hours per week."

---
## 1️⃣ Setup & Configuration

First, we'll set up the environment and load necessary dependencies.

In [14]:
# Import required libraries
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Import ADK components
from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.runners import Runner
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search, preload_memory
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService

print("✅ All imports loaded successfully")

✅ All imports loaded successfully


In [15]:
# Set up Gemini API key from environment variable
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
else:
    print("🔑 Authentication Error: 'GOOGLE_API_KEY' not found in .env file.")

✅ Gemini API key setup complete.


In [16]:
# Application constants
APP_NAME = "career_advisor"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

In [ ]:
async def auto_save_to_memory(callback_context):
    """
    Automatically save session to memory after each agent turn.

    This callback:
    - Extracts the agent name and session from the callback context
    - Saves the session to memory service
    - Logs success or failure
    - Fails gracefully without breaking the agent flow

    Args:
        callback_context: The callback context provided by ADK
    """
    try:
        agent_name = callback_context._invocation_context.agent.name
        session = callback_context._invocation_context.session

        await callback_context._invocation_context.memory_service.add_session_to_memory(session)

        print(f"💾 Saved {agent_name} output to memory")
    except Exception as e:
        print(f"⚠️  Warning: Failed to save to memory: {e}")
        

# Helper function for running sessions
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
    show_all_agents: bool = False,
):
    """Run queries in a session and display agent responses."""
    print(f"\n{'='*60}")
    print(f"Session: {session_name}")
    print(f"{'='*60}")

    app_name = runner_instance.app_name

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
        print("✅ New session created")
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
        print("✅ Existing session retrieved")

    if user_queries:
        if isinstance(user_queries, str):
            user_queries = [user_queries]

        for query in user_queries:
            print(f"\n👤 User: {query}")
            print(f"{'-'*60}")

            query_content = types.Content(role="user", parts=[types.Part(text=query)])

            responses = []

            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query_content
            ):
                if event.content and event.content.parts:
                    text = event.content.parts[0].text
                    if text and text != "None":
                        agent_name = getattr(event, 'author', 'Agent')

                        if show_all_agents:
                            print(f"\n🤖 {agent_name}:")
                            print(text)
                        else:
                            responses.append({'agent': agent_name, 'text': text})

            # If not showing all, print only final response
            if not show_all_agents and responses:
                last = responses[-1]
                print(f"\n🤖 {last['agent']}:")
                print(last['text'])
    else:
        print("⚠️  No queries provided!")

print("✅ Helper function defined.")

✅ Helper function defined.


In [ ]:
# Retry configuration for API calls
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Exponential backoff multiplier
    initial_delay=1,  # Initial delay in seconds
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

---
## 2️⃣ Agent Definitions

Now we'll define our two specialized agents that work together in sequence.

### Research Agent

The **Research Agent** is responsible for gathering information about career transitions.

**Tools:**
- `google_search`: Searches the web for relevant information
- `preload_memory`: Checks past conversations for context

**Output:** Structured research with:
- Key skills required
- Learning resources
- Market outlook
- Transition timeline
- Success stories

In [19]:
# 1. Research Agent
research_agent = LlmAgent(
    name="research_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a Research Agent specialized in career transitions.
    
    Your task:
    1. Extract career transition details from the user's message or check preload_memory for context
    2. Use the Google Search tool to research the target career path
    3. Focus on: required skills, typical career progression, salary ranges, and job market demand
    4. Search for: online courses, certifications, and learning resources
    5. Look for: success stories of people who made similar transitions
    
    Output format:
    ## Key Skills Required
    [List 5-7 essential skills with brief descriptions]
    
    ## Learning Resources
    [Specific courses, certifications, books, and platforms]
    
    ## Market Outlook
    [Job demand, salary ranges, growth trends with data]
    
    ## Transition Timeline
    [Typical timeframe for this transition based on the user's experience level]
    
    ## Success Stories
    [Brief examples of successful transitions]
    
    Keep your research comprehensive but concise. Focus on actionable information tailored to the user's background.
    """,
    tools=[google_search, preload_memory],
    output_key="research_summary", 
    after_agent_callback=auto_save_to_memory,
)

### Mentor Agent

The **Mentor Agent** creates personalized action plans based on research findings.

**Input:** Research summary from the Research Agent

**Output:** 3-phase transition plan with:
- **Phase 1 (Months 1-3)**: Foundation skills and quick wins
- **Phase 2 (Months 4-6)**: Portfolio building and networking
- **Phase 3 (Months 6-9)**: Job search preparation
- **Milestones**: Monthly goals and progress tracking
- **Leveraging Background**: How to use existing skills

In [20]:
# 2. Mentor Agent
mentor_agent = LlmAgent(
    name="mentor_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a Career Mentor Agent. Based on the research findings: {research_summary}
    
    Extract the user's context from the conversation history and research summary, then create a personalized, actionable transition plan.
    
    Structure your plan as follows:
    
    ## Phase 1: Foundation (Months 1-3)
    - Specific skills to learn first (prioritized based on their background)
    - Recommended courses/resources with links when available
    - Daily/weekly time commitment suggestions
    - Quick wins to build confidence
    
    ## Phase 2: Building Portfolio (Months 4-6)
    - Concrete projects to build (with examples relevant to their experience)
    - GitHub repositories to create
    - Communities to join (specific names)
    - Networking strategies leveraging their current role
    
    ## Phase 3: Job Search (Months 6-9)
    - Resume updates needed (specific sections)
    - Where to apply (companies, job boards)
    - Interview preparation tips
    - Portfolio presentation strategies
    
    ## Milestones & Checkpoints
    - Monthly goals with measurable outcomes
    - How to measure progress
    - Red flags and when to adjust course
    
    ## Leveraging Your Background
    - How to translate their existing skills to the new role
    - Unique advantages from their current position
    - How their experience is an asset
    
    Make it specific, realistic, and encouraging. Use actual course names, platforms, and communities when possible.
    Keep the tone supportive but practical - acknowledge challenges while emphasizing achievability.
    Tailor everything to their specific situation.
    """,
    output_key="career_advice",
)

---
## 3️⃣ Pipeline Assembly

Now we'll combine our agents into a sequential pipeline and create the runner.

In [21]:
# 3. Root Agent to orchestrate the workflow
root_agent = SequentialAgent(
    name="CareerPathPipeline",
    sub_agents=[research_agent, mentor_agent],
)

print("✅ Root agent created with 2-agent pipeline: Research -> Mentor")

✅ Root agent created with 2-agent pipeline: Research -> Mentor


### Initialize Runner

The **Runner** orchestrates the execution of our agent pipeline.

**Components:**
- `session_service`: Manages conversation history (short-term memory)
- `memory_service`: Stores knowledge across sessions (long-term memory)

Both agents in the pipeline will have access to these services.

In [22]:
# Initialize services
session_service = InMemorySessionService()  # Short-term conversation memory
memory_service = InMemoryMemoryService()    # Long-term knowledge storage

# Create the Runner
runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

print("✅ Runner initialized successfully!")
print(f"   Application: {APP_NAME}")
print(f"   User: {USER_ID}")
print(f"   Model: {MODEL_NAME}")

✅ Runner initialized successfully!
   Application: career_advisor
   User: default
   Model: gemini-2.5-flash-lite


---
## 4️⃣ Testing the Agent

Now let's test our career advisor with a sample query.

**What to include in your query:**
- Current role
- Years of experience
- Target role
- Time commitment (hours per week)

The agent will research the career path and create a personalized transition plan.

In [24]:
# Run the career advisor with a sample query
await run_session(
    runner,
    "I am a Product Manager with 5 years of experience. I want to transition to data science and can dedicate 10 hours per week.",
    session_name="career-transition-demo",
)


Session: career-transition-demo
✅ New session created

👤 User: I am a Product Manager with 5 years of experience. I want to transition to data science and can dedicate 10 hours per week.
------------------------------------------------------------
💾 Saved research_agent output to memory

🤖 mentor_agent:
It's fantastic that you're looking to leverage your 5 years of Product Management experience to transition into Data Science! Your background is a significant asset, as you already possess strong problem-solving, strategic thinking, and communication skills, which are highly valued in data science. With a dedicated 10 hours per week, a 1 to 2-year transition is absolutely achievable.

Here's a personalized plan to guide your journey:

## Phase 1: Foundation (Months 1-3)

The goal here is to build a solid understanding of the core technical skills. Given your PM background, you'll find the conceptual aspects of data analysis and problem-solving familiar, but the technical implementation